In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
import requests
import zipfile
import os
import io

#### Data Load

In [2]:
data_path = "./data/book_crossing/"

if not all(os.path.exists(data_path+file) for file in ["BX-Books.csv", "BX-Users.csv", "BX-Book-Ratings.csv"]):
    url = "http://www2.informatik.ler/BX/BX_CSV-Dump.zip"
    response = requests.get(url)
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))
    zip_file.extractall(data_path)
else:
    print("Data Available. Just Load It!")


books = pd.read_csv(data_path+"BX-Books.csv", sep=";", error_bad_lines=False, warn_bad_lines=False, encoding="latin-1")
users = pd.read_csv(data_path+"BX-Users.csv", sep=";", error_bad_lines=False, warn_bad_lines=False, encoding="latin-1")
ratings = pd.read_csv(data_path+"BX-Book-Ratings.csv", sep=";", error_bad_lines=False, warn_bad_lines=False, encoding="latin-1")


Data Available. Just Load It!


C:\Users\jmjhj\anaconda3\envs\rs\lib\site-packages\IPython\core\interactiveshell.py:3553: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\jmjhj\anaconda3\envs\rs\lib\site-packages\IPython\core\interactiveshell.py:3553: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\jmjhj\anaconda3\envs\rs\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [5]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [6]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


#### Preprocess the data

In [7]:
ratings = ratings.merge(books[["ISBN", "Book-Title"]], on="ISBN")
ratings = ratings.drop(['ISBN'], axis=1)

In [8]:
ratings

,User-ID,Book-Rating,Book-Title
0,276725,0,Flesh Tones: A Novel
1,2313,5,Flesh Tones: A Novel
2,6543,0,Flesh Tones: A Novel
3,8680,5,Flesh Tones: A Novel
4,10314,9,Flesh Tones: A Novel
...,...,...,...
1031131,276688,0,Mostly Harmless
1031132,276688,7,Gray Matter
1031133,276690,0,Triplet Trouble and the Class Trip (Triplet Tr...
1031134,276704,0,A Desert of Pure Feeling (Vintage Contemporaries)


#### Filtering

In [9]:
# Filter books with at least min_book_ratings ratings
min_book_ratings = 10
book_rating_counts = ratings['Book-Title'].value_counts()
filtered_books = book_rating_counts[book_rating_counts >=min_book_ratings].index
ratings = ratings[ratings['Book-Title'].isin(filtered_books)]

In [10]:
# Filter users who rated at least min_user_ratings books
min_user_ratings = 5
user_rating_counts = ratings['User-ID'].value_counts()
filtered_users = user_rating_counts[user_rating_counts >=min_user_ratings].index
ratings = ratings[ratings['User-ID'].isin(filtered_users)]

#### Create the user-item matrix

In [11]:
user_item_matrix = ratings.pivot_table(index = "User-ID", columns="Book-Title", values="Book-Rating").fillna(0)
user_item_matrix

Book-Title,"Earth Prayers From around the World: 365 Prayers, Poems, and Invocations for Honoring the Earth","Good Wives: Image and Reality in the Lives of Women in Northern New England, 1650-1750",Murder of a Sleeping Beauty (Scumble River Mysteries (Paperback)),"Q-Space (Star Trek The Next Generation, Book 47)","Q-Zone (Star Trek The Next Generation, Book 48)",!Yo!,'Salem's Lot,...AND THE HORSE HE RODE IN ON : THE PEOPLE V. KENNETH STARR,01-01-00: A Novel of the Millennium,01-01-00: The Novel of the Millennium,...,"\O\"" Is for Outlaw""","\Surely You're Joking, Mr. Feynman!\"": Adventures of a Curious Character""","\The Happy Prince\"" and Other Stories (Penguin Popular Classics)""","\What Do You Care What Other People Think?\"": Further Adventures of a Curious Character""",e,iI Paradiso Degli Orchi,murder@maggody.com : An Arly Hanks Mystery (Arly Hanks Mysteries (Paperback)),one hundred years of solitude,stardust,why I'm like this : True Stories
User-ID,,,,,,,,,,,,,,,,,,,,,
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278723,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278771,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Compute cosine similarity matrix

In [12]:
cosine_sim_matrix = cosine_similarity(user_item_matrix)
cosine_sim_matrix # 14322명의 user들이 평점을 주었던 pattern에 의해서

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [13]:
cosine_sim_matrix.shape

(14322, 14322)

In [14]:
cosine_sim_matrix.mean(axis=0)

array([0.00019009, 0.00032347, 0.00117289, ..., 0.00555277, 0.000628  ,
       0.00123018])

#### split user indices into train and test sets
- 지금, 엄밀하게 train : test = 8 : 2 이렇게 나누면 의미 없음
    - 이미 사용함...

In [15]:
user_indices = np.arange(user_item_matrix.shape[0])
train_user_indices, test_user_indices = train_test_split(user_indices, test_size=0.2, random_state=42)

#### recommendation

In [16]:
def collaborative_filtering_recommendation(user_index, k=10):
    user_similarity = cosine_sim_matrix[user_index]
    top_k_similar_users = np.argsort(user_similarity)[-k-1:-1][::-1] # 오름차순으로 해주는거니까..
    similarity_user_ratings = user_item_matrix.iloc[top_k_similar_users] # 10 x 17446
    mean_ratings = similarity_user_ratings.mean(axis=0) # shape : 17446
    top_k_books = mean_ratings.sort_values(ascending=False).head(k).index
    return top_k_books

In [17]:
def recoomend_books(user_index, strategy, k=10):
    if strategy == "collaborative":
        top_k_books = collaborative_filtering_recommendation(user_index, k=k)
    elif strategy == "random":
        top_k_books = random_recommendation(ratings, n=k)
    elif strategy == "popularity":
        top_k_books = popularity_recommendation(ratings, n=k)
    else:
        raise ValueError("Invalid recommendation strategy")
    
    return top_k_books


#### Evaluation

In [18]:
def evaluate_model(strategy, k=10):
    true_positive = 0
    false_positive = 0
    false_negative = 0
    
    for user_index in test_user_indices:
        true_books = set(user_item_matrix.iloc[user_index][user_item_matrix.iloc[user_index] > 0].index)
        # user_item_matrix.iloc[user_index] > 0 : 평점을 준 책들 
        recommended_books = set(recoomend_books(user_index, strategy, k))
        
        tp = len(true_books.intersection(recommended_books))
        fp = len(recommended_books - true_books)
        fn = len(true_books - recommended_books)
        
        true_positive += tp
        false_positive += fp
        false_negative += fn
    
    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)
    
    return precision, recall

In [19]:
collaborative_precision, collaborative_recall = evaluate_model(strategy="collaborative")
print(collaborative_precision, collaborative_recall)

0.31095986038394413 0.2601395742692791
